In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

In [2]:
d3 = pd.read_csv(r"E:\Learning\TEAI Cup\Data\Main Data\Metadata_L2.csv", sep=";", encoding='latin1')
d3

,qcpc_event_id,qcpc_event_machine_id,qcpc_event_machine_status_active_code,qcpc_event_create_timestamp,qcpc_event_machine_module_description,qcpc_event_part_number,qcpc_event_error_text,qcpc_event_machine_stop_code
0,27601227596,S-276,200,2025-12-08 23:33:41,Kamera,7-1452668-3,I-Kasten SP2: Cur= 1.421 LTL= 1.780 UTL= 1.880,0
1,27601226196,S-276,200,2025-12-05 11:30:08,Kamera,7-1452668-3,I-Kasten SP2: Cur= 1.895 LTL= 1.780 UTL= 1.880,0
2,26803927805,S-268,30000,2025-12-06 10:33:21,Bruderer,2-1703930-2,WERKZEUGSICHERUNG: KANAL7 STOERUNG,0
3,26903485075,S-269,30000,2025-12-08 21:48:34,Bruderer,2-1703930-1,WERKZEUGSICHERUNG: KANAL7 STOERUNG,0
4,22600234145,S-226,200,2025-12-08 07:13:26,Kamera,5-965906-1,Teil 1 Gap Y: Cur= 0.530 LTL= 0.470 UTL= 0.530,0
...,...,...,...,...,...,...,...,...
9910,27601228404,S-276,200,2025-12-09 04:49:32,Kamera,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 53.874 LTL= 0....,0
9911,26803918960,S-268,30000,2025-12-05 09:58:03,Bruderer,2-1703930-2,WERKZEUGSICHERUNG: KANAL7 STOERUNG,0
9912,26903485594,S-269,200,2025-12-09 13:37:47,Kamera,2-1703930-1,Spur 2 Iso: Cur= Nicht messbar LTL= 1.800 UTL...,0
9913,27601228836,S-276,200,2025-12-09 18:37:08,Kamera,7-1452668-3,SP 1 durchgebrannt GW 70: Cur= 51.532 LTL= 0....,0


In [3]:
# Maybe this data doesnt requires filtering just requires to be analysed